# Introduzione a LangChain

***Argomenti:***
* Runnable
* Prompt Template
* Memoria Sequenziale
* Parserizzazione di Input e Output

In [1]:
import os
OPENAI_KEY = os.getenv("openai_key")

In [2]:
from langchain_openai import ChatOpenAI   # pip install langchain-openai

model = ChatOpenAI(
    openai_api_key=OPENAI_KEY, 
    model_name = "gpt-4o-mini",
    temperature=0.7, #livello di creatività/casualità nelle risposte (0: minima variabilità, 1: massima variabilità)
    max_tokens=1024, #numero massimo di token che il modello può generare in una sola risposta
    request_timeout=30,
    # ---
    # Possiamo usare questo connettore anche per modelli in locale
    # ---
    #model="meta-llama/Llama-3.2-3B-Instruct", 
    #base_url = "http://127.0.0.1:8000/v1",
)

# Le info sui pricing dei modelli si possono trovare qui: 
# https://platform.openai.com/docs/pricing

C:\Cloud\GitHub\LLMs_MasterClass\lc1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain.chat_models import init_chat_model

#  init_chat model

model = init_chat_model(
    "openai:gpt-4o-mini",
    openai_api_key=os.getenv("openai_key"),
    temperature=0.7,
)

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRXuDvlNDmDGF5QwPETEs3eh7RHNGmKBpgwyw&s">  
  
# OpenAI GPT-4o mini
**Generative Pre-trained Transformer 4o mini** (GPT-4o mini) è una versione compatta della serie di modelli GPT-4, sviluppata da OpenAI nel 2024. 

Il modello GPT-4o mini è stato progettato per bilanciare efficienza e prestazioni, offrendo una capacità inferiore rispetto ai modelli completi di GPT-4, ma con una velocità e un costo migliorati per applicazioni che richiedono risposte rapide e consumo limitato di risorse. Ideale per implementazioni su larga scala in ambienti con restrizioni computazionali o per casi d'uso che non necessitano della massima potenza offerta dai modelli più grandi.

*Caratteristiche:*
* Contesto di 128K tokens
* Output di 16384 token massimo
* Ottimizzato per l'efficienza in ambienti a risorse limitate
* Addestrato con tecniche di RLHF (Reinforcement Learning from Human Feedback)
* Eccellente per applicazioni che richiedono un bilanciamento tra costo, velocità e prestazioni

#### Reinforcement Learning from Human Feedback

Nel **RLHF (Reinforcement Learning from Human Feedback)** il modello genera delle risposte che vengono valutate poi dagli esseri umani, che forniscono un feedback di qualità (es. preferenze su quale risposta è migliore). Questo feedback viene trasformato in "ricompense" tramite un modello di ricompensa (similmente a come accade nel Reinforcement Learning), che assegna punteggi alle risposte generate dall'AI in base a quanto si avvicinano alle preferenze umane.

Gli algoritmi di apprendimento per rinforzo utilizzano questi punteggi di ricompensa per aggiornare i pesi del modello in modo iterativo. A differenza del fine-tuning statico, il RLHF consente al modello di evolversi e migliorare continuamente, generando risposte sempre più allineate con i valori e le preferenze umane, seguendo un processo di adattamento graduale.

<hr>    
<img src="imgs/seq_memory.png" width="800">    
    
[https://python.langchain.com/docs/concepts/messages/#langchain-messages](https://python.langchain.com/docs/concepts/messages/#langchain-messages)

In [4]:
# LangChain si occupa autonomamente di gestire i vari tag (<SOS>, <EOS>, etc) corretti per il modello che stiamo utilizzano
# ---
# In questo caso il nostro output sarà un AIMessage che conterrà un content e una serie di metadati associati
# Tutti i messaggi hanno sempre la chiave content da cui recuperarci il corpo del messaggio
model.invoke("ciao sono Gennaro")

AIMessage(content='Ciao Gennaro! Come posso aiutarti oggi?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 13, 'total_tokens': 25, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CWLQtrd6Jmssu1HjrAc8S0mj99pbZ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--97580cc3-bb4e-4445-bced-888481a18ed3-0', usage_metadata={'input_tokens': 13, 'output_tokens': 12, 'total_tokens': 25, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

<hr>    

<img src="imgs/prompt_template.png" width="800">    
    
[https://python.langchain.com/docs/how_to/#prompt-templates](https://python.langchain.com/docs/how_to/#prompt-templates)

In [5]:
# prompt template

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate([
    ("system", "Act as a world class Machine Learning engineer. Use italian language. Ends your answers with a reference to the beauty of using data science in any decision you make."),
    ("user", "{pippo}"),
])

# concatenazione del prompt al modello
chain = prompt | model

## Interfaccia Runnable

Per rendere più semplice la creazione di catene di eventi/esecuzione anche molto complesse i componenti di LangChain implementano tutti un protocollo "runnable" tramite un'interfaccia comune che permette di usare qualsiasi componente in modo standard; di seguito sono elencati i 3 principali metodi:

* **stream** - inviare risposte parziali mentre vengono generate
* **invoke** - eseguire la catena su un input
* **batch** - esecuzione della catena su più input

Uno dei vantaggi delle interfacce Runnable è dato dal fatto che dei componenti *runnable* possono essere concatenati in sequenze di esecuzione, facendo in modo che, automaticamente, gli output di un componente possano entrare in input ad un altro; il comando *pipe* `|` serve a questo e permette, nella sintassi LCEL (LangChain Expression Language) di creare componenti runnable partendo da altri componenti runnable, configurandoli in una sequenza di componenti che agiranno sinergicamente.

In [6]:
prompt.invoke({"pippo": "elencami i primi 3 pianeti del sistema solare"})

ChatPromptValue(messages=[SystemMessage(content='Act as a world class Machine Learning engineer. Use italian language. Ends your answers with a reference to the beauty of using data science in any decision you make.', additional_kwargs={}, response_metadata={}), HumanMessage(content='elencami i primi 3 pianeti del sistema solare', additional_kwargs={}, response_metadata={})])

In [7]:
chain.invoke({"pippo": "elencami i primi 3 pianeti del sistema solare"})

AIMessage(content="I primi tre pianeti del sistema solare, partendo dal sole, sono:\n\n1. Mercurio\n2. Venere\n3. Terra\n\nQuesti pianeti presentano caratteristiche uniche e affascinanti. La bellezza di esplorare l'universo, così come l'analisi dei dati, ci permette di prendere decisioni più informate e di apprezzare la complessità del nostro universo.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 56, 'total_tokens': 144, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CWLR1S5nwk7nGdSNYy7yvM0y4CeWr', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--8b552260-59cf-4138-a379-377313e310c1-0', usage_metadata={'in

In [9]:
# streaming degli output

answer = None
for chunk in model.stream("elencami i primi 3 pianeti del sistema solare"):
    answer = chunk if answer is None else answer + chunk
    print(answer.text, "\n\n     -----\n\n")

print("-" * 10, "\n", answer.content)

 

     -----


I 

     -----


I primi 

     -----


I primi tre 

     -----


I primi tre pian 

     -----


I primi tre pianeti 

     -----


I primi tre pianeti del 

     -----


I primi tre pianeti del sistema 

     -----


I primi tre pianeti del sistema sol 

     -----


I primi tre pianeti del sistema solare 

     -----


I primi tre pianeti del sistema solare, 

     -----


I primi tre pianeti del sistema solare, in 

     -----


I primi tre pianeti del sistema solare, in ordine 

     -----


I primi tre pianeti del sistema solare, in ordine di 

     -----


I primi tre pianeti del sistema solare, in ordine di distanza 

     -----


I primi tre pianeti del sistema solare, in ordine di distanza dal 

     -----


I primi tre pianeti del sistema solare, in ordine di distanza dal Sole 

     -----


I primi tre pianeti del sistema solare, in ordine di distanza dal Sole, 

     -----


I primi tre pianeti del sistema solare, in ordine di distanza dal Sole, sono 

   

In [12]:
# gestione degli eventi durante lo streaming

async for event in model.astream_events("elencami i primi 3 pianeti del sistema solare"):

    if event["event"] == "on_chat_model_start":
        print(f"Input: {event['data']['input']}\n\n")

    elif event["event"] == "on_chat_model_stream":
        print(f"Token: {event['data']['chunk'].text}")

    elif event["event"] == "on_chat_model_end":
        print(f"\n\nFull message:\n{event['data']['output'].text}")

    else:
        pass

Input: elencami i primi 3 pianeti del sistema solare


Token: 
Token: I
Token:  primi
Token:  tre
Token:  pian
Token: eti
Token:  del
Token:  sistema
Token:  sol
Token: are
Token: ,
Token:  part
Token: endo
Token:  dal
Token:  Sole
Token: ,
Token:  sono
Token: :


Token: 1
Token: .
Token:  **
Token: Merc
Token: urio
Token: **

Token: 2
Token: .
Token:  **
Token: Ven
Token: ere
Token: **

Token: 3
Token: .
Token:  **
Token: Ter
Token: ra
Token: **
Token:  


Token: Se
Token:  hai
Token:  bisogno
Token:  di
Token:  ulterior
Token: i
Token:  informazioni
Token:  su
Token:  cias
Token: cun
Token:  pian
Token: eta
Token: ,
Token:  fam
Token: m
Token: elo
Token:  sapere
Token: !
Token: 
Token: 
Token: 


Full message:
I primi tre pianeti del sistema solare, partendo dal Sole, sono:

1. **Mercurio**
2. **Venere**
3. **Terra** 

Se hai bisogno di ulteriori informazioni su ciascun pianeta, fammelo sapere!


In [13]:
# esempio di chiamate in batch di più richieste
# da servire in parallelo (quando il sistema lo consente)
# con verifica del tempo di esecuzione

import time

queries = [
    "elencami i primi 3 pianeti del sistema solare",
    "elencami gli ultimi 3 pianeti del sistema solare",
    "elenca tre pianeti del sistema solare"
]

start_batch = time.time()
responses_batch = model.batch(queries)
end_batch = time.time()

print("=== Risultati batch ===")
for r in responses_batch:
    print(r.content)
print(f"\nTempo totale batch: {end_batch - start_batch:.3f} secondi\n\n")

responses_single = []
start_single = time.time()
for q in queries:
    resp = model.invoke(q)
    responses_single.append(resp)
end_single = time.time()

print("=== Risultati singoli ===")
for r in responses_single:
    print(r.content)
print(f"\nTempo totale chiamate singole: {end_single - start_single:.3f} secondi")

=== Risultati batch ===
I primi tre pianeti del sistema solare, in ordine di distanza dal Sole, sono:

1. **Mercurio**
2. **Venere**
3. **Terra**

Se hai bisogno di ulteriori informazioni su ciascun pianeta, fammi sapere!
I tre pianeti più esterni del sistema solare, partendo dal Sole, sono:

1. **Uranus** - Il settimo pianeta del sistema solare, noto per il suo colore blu-verde dovuto alla presenza di metano nell'atmosfera e per il suo asse di rotazione fortemente inclinato.

2. **Nettuno** - L'ottavo pianeta, spesso descritto come il pianeta più lontano dal Sole. Anch'esso ha una colorazione blu, sempre a causa del metano nell'atmosfera, e presenta intense tempeste atmosferiche.

3. **Plutone** - Sebbene non sia più considerato un pianeta principale (è stato riclassificato come pianeta nano nel 2006), Plutone è spesso incluso nelle discussioni sui pianeti del sistema solare. Si trova oltre Nettuno e ha un'orbita altamente ellittica.

Se hai bisogno di ulteriori informazioni su ciascu

In [21]:
# restituzione di un output appena pronto,
# durante l'esecuzione di più input in batch

start_time = time.time()
for response in model.batch_as_completed([
    "Descrivi brevemente La Divina Commedia",
    "2 + 2 = ?",
    "Ciao, sono Enzo"
], config={
        'max_concurrency': 5,  # numero limite di chiamate in parallelo - inutile in questo esempio
    }):
    print(response[0], time.time() - start_time, "\n", response[1].content, "\n\n")

1 0.8644392490386963 
 2 + 2 = 4. 


2 1.201648473739624 
 Ciao Enzo! Come posso aiutarti oggi? 


0 7.384363412857056 
 "La Divina Commedia" è un poema epico scritto da Dante Alighieri nel XIV secolo, considerato uno dei capolavori della letteratura mondiale. L'opera è composta da tre cantiche: Inferno, Purgatorio e Paradiso, per un totale di 100 canti. 

La trama segue il viaggio del protagonista, Dante stesso, attraverso questi tre regni dell'aldilà, guidato inizialmente dal poeta romano Virgilio e, successivamente, da Beatrice, simbolo dell'amore divino. 

Nel "Inferno", Dante esplora i peccati e le punizioni dei dannati, descrivendo i vari cerchi dell'Inferno e le anime che vi si trovano. Nel "Purgatorio", si affronta il tema della redenzione e della purificazione delle anime in attesa di entrare in Paradiso. Infine, nel "Paradiso", Dante descrive la beatitudine dei giusti e l'unione con Dio.

L'opera affronta temi profondi come la giustizia, la fede, l'amore e la ricerca della ve

### Ovviamente non siamo limitati ad un solo placeholder, ma possiamo usarne più di uno

In [17]:
prompt = ChatPromptTemplate([
    ("system", "Act as a useful AI Assistant. Always answer the user question using only the {language} language, regardless of the language used by the user"),
    ("user", "{query}"),
])

# concatenazione del prompt al modello
chain = prompt | model

In [18]:
chain.invoke({"language":"italian", "query": "Elencami i primi 3 pianeti del sistema solare"})

AIMessage(content='I primi tre pianeti del sistema solare, in ordine di distanza dal Sole, sono Mercurio, Venere e Terra.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 50, 'total_tokens': 76, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CW31JHk7Cmu7GCBot7BQjsrV5KNCF', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--5e96ea69-3ba0-40c3-9838-b83c2b030999-0', usage_metadata={'input_tokens': 50, 'output_tokens': 26, 'total_tokens': 76, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [19]:
chain.invoke({"language":"english", "query": "Elencami i primi 3 pianeti del sistema solare"})

AIMessage(content='The first three planets of the solar system are Mercury, Venus, and Earth.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 50, 'total_tokens': 66, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CW31K7zfmuRxvXLAbAjQMiNhaMcuL', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--eeaae372-83ef-44aa-b289-e34d61bca934-0', usage_metadata={'input_tokens': 50, 'output_tokens': 16, 'total_tokens': 66, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

##  Memoria Sequenziale

Ovviamente in questo caso il modello non ha modo di accedere alla memoria della nostra conversazione.

In [20]:
prompt = ChatPromptTemplate([
    ("system", "Act as a useful AI Assistant."),
    ("user", "{query}"),
])

# concatenazione del prompt al modello
chain = prompt | model

In [21]:
chain.invoke({"query": "Ciao, io mi chiamo Simone"})

AIMessage(content='Ciao Simone! Come posso aiutarti oggi?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 26, 'total_tokens': 36, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CW31OSZPVKjbPtqXTBCoPUuDzGIRV', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--36ec627b-8272-495d-91d6-a74bb0af4f65-0', usage_metadata={'input_tokens': 26, 'output_tokens': 10, 'total_tokens': 36, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [22]:
chain.invoke({"query": "Come mi chiamo?"})

AIMessage(content='Non ho accesso a informazioni personali, quindi non posso sapere come ti chiami. Posso aiutarti in altro modo?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 23, 'total_tokens': 48, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CW31RcfHaFhJ95h8gashf889KcRWr', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--4a2b596e-0614-4daa-a793-d474344239a4-0', usage_metadata={'input_tokens': 23, 'output_tokens': 25, 'total_tokens': 48, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

 Per permettere al modello di "ricordare" informazioni dalla conversazione, bisogna introdurre il concetto di __memoria__
 
Aggiungere la memoria significa, a conti fatti, iniettare lo storico della conversaione avuta sino a quel momento all'interno del prompt, così che ad ogni nuova richiesta che facciamo al LLM, questo riceva anche tutto lo scambio di conversazioni avute oltre che al SystemMessage e all'ultimo messaggio dell'utente

In [23]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts  import MessagesPlaceholder 

# -----------------------------------------------
# A regime il nostro prompt sarà strutturato così:
# > System message
# > History Message 1
# > History Message 2
# .
# .
# .
# > History Message N
# > Human Message (in input)
# -----------------------------------------------
prompt = ChatPromptTemplate([
    ("system", "Act as a useful AI Assistant. Answer using italian language"),
    MessagesPlaceholder(variable_name="history"),
    ("user", "{query}"),
])

chain = prompt | model

Facciamo una prova con una lista di messaggi fittizi che simulano uno storico di conversazione

In [24]:
dummy_messages = [
   ("user", "Ciao, io sono Simone"),
   ("assistant", "Ciao Simone, come posso aiutarti?"),
   ("user", "Volevo chiederti una cosa, posso?"),
   ("assistant", "Certamente! Dimmi pure"),
]

prompt.invoke({"history":dummy_messages, "query": "elencami i primi 3 pianeti del sistema solare"})

ChatPromptValue(messages=[SystemMessage(content='Act as a useful AI Assistant. Answer using italian language', additional_kwargs={}, response_metadata={}), HumanMessage(content='Ciao, io sono Simone', additional_kwargs={}, response_metadata={}), AIMessage(content='Ciao Simone, come posso aiutarti?', additional_kwargs={}, response_metadata={}), HumanMessage(content='Volevo chiederti una cosa, posso?', additional_kwargs={}, response_metadata={}), AIMessage(content='Certamente! Dimmi pure', additional_kwargs={}, response_metadata={}), HumanMessage(content='elencami i primi 3 pianeti del sistema solare', additional_kwargs={}, response_metadata={})])

Ovviamente in un contesto reale, non mettiamo noi manualmente i messaggi in una lista, ma lasciamo che sia LangChain a farlo per noi.
In questo modo abbiamo anche la possibilità di tenere separate delle sessioni di conversazioni, ognuna con la sua memoria univoca

### RunnableWithMessageHistory

**Runnable che gestisce la cronologia dei messaggi di chat per un altro Runnable.**

Una *chat message history* è una sequenza di messaggi che rappresenta una conversazione.

`RunnableWithMessageHistory` incapsula un altro `Runnable` e gestisce per questo la *chat message history*; è responsabile della lettura e dell'aggiornamento della cronologia dei messaggi.

Deve sempre essere invocato con una configurazione (`config`) che contenga i parametri appropriati per il recupero dei messaggi corretti.

Per impostazione predefinita, il `Runnable` si aspetta un singolo parametro di configurazione chiamato `session_id` di tipo stringa; questo parametro viene utilizzato per creare una nuova *chat message history* o recuperarne una esistente associata al `session_id` specificato.


In [25]:
chat_map = {}

def get_chat_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in chat_map:
        chat_map[session_id] = InMemoryChatMessageHistory()
    return chat_map[session_id]


chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history"
)

In [26]:
chain_with_history.invoke(
    {"query": "Ciao, mi chiamo Simone"},
    config={"session_id": "id_1234"}
)

AIMessage(content='Ciao Simone! Come posso aiutarti oggi?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 29, 'total_tokens': 39, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CW31giABfqdjBxYQMfjcCRj7hwL0C', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--ea5ba668-6e2c-4863-ae58-5c07416dbc2c-0', usage_metadata={'input_tokens': 29, 'output_tokens': 10, 'total_tokens': 39, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [27]:
chat_map["id_1234"]

InMemoryChatMessageHistory(messages=[HumanMessage(content='Ciao, mi chiamo Simone', additional_kwargs={}, response_metadata={}), AIMessage(content='Ciao Simone! Come posso aiutarti oggi?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 29, 'total_tokens': 39, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CW31giABfqdjBxYQMfjcCRj7hwL0C', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--ea5ba668-6e2c-4863-ae58-5c07416dbc2c-0', usage_metadata={'input_tokens': 29, 'output_tokens': 10, 'total_tokens': 39, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})])

In [28]:
chain_with_history.invoke(
    {"query": "Ciao, mi chiamo Enzo"},
    config={"session_id": "id_5678"}
)

AIMessage(content='Ciao Enzo! Come posso aiutarti oggi?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 30, 'total_tokens': 41, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CW31jI4RdhSwDe7mLoIbNKzXpFUvU', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--f1b40f4d-2241-4190-8480-ced116b387a7-0', usage_metadata={'input_tokens': 30, 'output_tokens': 11, 'total_tokens': 41, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [29]:
chat_map["id_5678"]

InMemoryChatMessageHistory(messages=[HumanMessage(content='Ciao, mi chiamo Enzo', additional_kwargs={}, response_metadata={}), AIMessage(content='Ciao Enzo! Come posso aiutarti oggi?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 30, 'total_tokens': 41, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CW31jI4RdhSwDe7mLoIbNKzXpFUvU', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--f1b40f4d-2241-4190-8480-ced116b387a7-0', usage_metadata={'input_tokens': 30, 'output_tokens': 11, 'total_tokens': 41, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})])

In [30]:
chain_with_history.invoke(
    {"query": "Come mi chiamo?" },
    config={"session_id": "id_1234"}
)

AIMessage(content="Ti chiami Simone. Hai bisogno di aiuto con qualcos'altro?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 52, 'total_tokens': 68, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CW31kuaz0QuWFbgeaoZ4rwIB3x5Tx', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--5254824f-8eb3-4f44-817a-ce1e99c73d22-0', usage_metadata={'input_tokens': 52, 'output_tokens': 16, 'total_tokens': 68, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [31]:
chain_with_history.invoke(
    {"query": "Come mi chiamo?" },
    config={"session_id": "id_5678"}
)

AIMessage(content='Ti chiami Enzo! Come posso assisterti?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 54, 'total_tokens': 65, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CW31lw3Shcyb5WhHTPX4lptbFdWmr', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--1e585b60-40a8-41f4-aaf7-ce6149678148-0', usage_metadata={'input_tokens': 54, 'output_tokens': 11, 'total_tokens': 65, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# Prompt Template
Negli esempi sopra abbiamo sempre visto l'utilizzo di un ChatPromptTemplate, ma possiamo anche utilizzare un oggetto PromptTemplate, che funziona in maniera molto simile

In [32]:
from langchain_core.prompts import PromptTemplate

template = """Agisci come un esperto Data Scientist rispondendo a ogni input con riferimenti alla bellezza della Data Science.
Qui l'input: {input}
"""

prompt = PromptTemplate.from_template(template)

chain = prompt | model 

chain.invoke({"input": "Quali sono i primi 3 pianeti del Sistema Solare?"})

AIMessage(content="La Data Science ci offre strumenti straordinari per esplorare e analizzare l'universo, e anche le informazioni apparentemente semplici possono rivelare connessioni affascinanti. I primi tre pianeti del Sistema Solare, in ordine di distanza dal Sole, sono Mercurio, Venere e Terra. \n\nImmagina di utilizzare l'analisi dei dati per confrontare le caratteristiche di questi pianeti: la temperatura, la composizione atmosferica e le condizioni di superficie. Attraverso visualizzazioni dei dati, possiamo creare grafici che mostrano come ciascun pianeta si distingue per le sue peculiarità, rivelando non solo la loro bellezza scientifica, ma anche le storie che i dati possono raccontare. La Data Science ci permette di vedere oltre le informazioni superficiali, esplorando la complessità e l'interconnessione dell'universo.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 184, 'prompt_tokens': 50, 'total_tokens': 234, 'completion_toke

# Parserizzazione degli Output

I parser sono degli elementi di LangChain che possiamo utilizzare per trasformare l’output grezzo di un modello in un formato strutturato e più facilmente utilizzabile nel nostro codice.

Quando un LLM restituisce una risposta, questa può essere passata attraverso un parser che può aiutarci, tra le varie cose a interpretare e validare quell’output, estrarre informazioni chiave, trasformare il testo in dizionari, oggetti JSON, liste, o classi Python, etc

Esistono diverse tipologie di Parser che possiamo utilizzare a seconda del tipo di output atteso e del livello di struttura che vogliamo ottenere

### String Output Parser

In [33]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"input": "Quali sono i primi 3 pianeti del Sistema Solare?"})

'La bellezza della Data Science risiede nella sua capacità di trasformare dati grezzi in conoscenza significativa, proprio come l\'astronomia trasforma l\'osservazione dei corpi celesti in comprensione dell\'universo. Ora, per rispondere alla tua domanda, i primi tre pianeti del Sistema Solare, partendo dal Sole, sono:\n\n1. **Mercurio**: Il pianeta più vicino al Sole, un corpo roccioso con temperature estreme.\n2. **Venere**: Spesso chiamato "il gemello della Terra" per le sue dimensioni simili, ma con un\'atmosfera molto densa e calda.\n3. **Terra**: Il nostro pianeta, unico nel suo genere per la vita e la biodiversità.\n\nLa Data Science può aiutarci a esplorare non solo le caratteristiche di questi pianeti, ma anche a modellare e prevedere fenomeni astrali attraverso l\'analisi dei dati provenienti da telescopi e sonde spaziali. Ogni dato raccolto è un pezzo del puzzle che ci avvicina alla comprensione del nostro universo.'

### Lista di valori

In [34]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="{query}.\n{format_instructions}",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions},
)

In [35]:
print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [36]:
prompt.invoke({"query": "elenca i pianeti del sistema solare in ordine dal più vicino al più lontano dal Sole"}).text

'elenca i pianeti del sistema solare in ordine dal più vicino al più lontano dal Sole.\nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [37]:
chain = prompt | model 
res = chain.invoke({"query": "elenca i pianeti del sistema solare in ordine dal più vicino al più lontano dal Sole"})

print( type(res) )
print("---")
print(res)

print("\n===\n")

print( type(res.content) )
print("---")
print(res.content)

<class 'langchain_core.messages.ai.AIMessage'>
---
content='Mercurio, Venere, Terra, Marte, Giove, Saturno, Urano, Nettuno' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 56, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CW32L7EvPIFOs0MHqFOSd3qywavet', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--4159dffa-6ca5-4bba-8fae-2653792f1c78-0' usage_metadata={'input_tokens': 56, 'output_tokens': 21, 'total_tokens': 77, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}

===

<class 'str'>
---
Mercurio, Venere, Terra, Marte, Giove, Saturno,

In [38]:
chain = prompt | model | output_parser
res = chain.invoke({"query": "elenca i pianeti del sistema solare in ordine dal più vicino al più lontano dal Sole"})

print(type (res) )
print("---")
print(res)

<class 'list'>
---
['Mercurio', 'Venere', 'Terra', 'Marte', 'Giove', 'Saturno', 'Urano', 'Nettuno']


## Output strutturato in LangChain 1.0

In molti casi, quando utilizzi un modello di linguaggio all’interno di una pipeline (ad esempio per chatbot, agent, estrazione dati, RAG, ecc.), vuoi che l’output non sia semplicemente un blocco di testo libero, ma sia **organizzato** in una forma nota e prevedibile (dictionary, JSON, oggetto Pydantic…).
Questo è ciò che si intende per *output strutturato*.
La libreria LangChain 1.0 introduce un supporto formale per tale modalità, mediante il metodo `with_structured_output()` e/o parser dedicati.

### Perché usare l’output strutturato

* Permette di definire uno **schema** (campi, tipi, descrizioni) che l’output dovrà rispettare.
* Favorisce l’integrazione downstream (es. inserimento in database, orditura di oggetti, logica condizionale) in modo programmatico, anziché dover parsare manualmente un testo libero.
* Aumenta l’affidabilità del sistema: meno “interpretazione libera” del modello, più predicibilità del formato.
* In LangChain 1.0 è possibile sfruttare modelli che supportano direttamente modalità strutturate (come JSON mode, funzione/tool-calling) grazie a `with_structured_output()`.


### Differenza rispetto al semplice “parser di output”

Il “semplice parser di output” (output parser) in LangChain pone un approccio diverso: invece di far generare al modello già un output *certamente* conforme allo schema, si prende un output di testo libero (o semi-noto) e lo si “parsifica” successivamente. Alcuni punti chiave:

* Con un output parser classico (es. `JsonOutputParser`, `StructuredOutputParser`, `PydanticOutputParser`) definisci delle istruzioni (via `get_format_instructions()`) che instradano il modello a generare un certo formato sfruttando il prompt di input, ma non hai garanzia totale che il modello rispetti lo schema.
* Questo approccio è più “flessibile” ma anche più fragile: se il modello devia dal formato richiesto (ad esempio aggiunge testo narrativo, non restituisce JSON valido…) allora il parsing può fallire.
* Al contrario, con `with_structured_output()`, quando il modello provider lo supporta, l’output viene generato *già* come struttura conforme (grazie a tool calling, JSON mode o schema nativo) e quindi il downstream è più solido.

### Tabella comparativa (concettuale)

| Approccio                     | Dove si applica                          | Punti di forza                      | Limiti                                                     |
| ----------------------------- | ---------------------------------------- | ----------------------------------- | ---------------------------------------------------------- |
| Parser di output classico     | Qualsiasi modello, testo libero          | Alta compatibilità                  | Fragile: parsing manuale, rischio errori                   |
| Output strutturato con schema | Modelli + provider che supportano schema | Maggiore affidabilità, tipizzazione | Richiede supporto del modello/provider, schema da definire |

------

Con LangChain 1.0 l’output strutturato è la modalità consigliata quando si ha bisogno di output prevedibile, di un particolare formato e facilmente integrabile con sistemi esterni.
Il semplice parser resta utile in scenari più “liberi” ma comporta maggiore rischio di deviazioni e fallimenti.

### JSON 

In [43]:
import json

json_schema = {
    "title": "Movie",
    "description": "A movie with details",
    "type": "object",
    "properties": {
        "title": {
            "type": "string",
            "description": "The title of the movie"
        },
        "year": {
            "type": "integer",
            "description": "The year the movie was released"
        },
        "director": {
            "type": "string",
            "description": "The director of the movie"
        },
        "rating": {
            "type": "number",
            "description": "The movie's rating out of 10"
        }
    },
    "required": ["title", "year", "director", "rating"]
}

model_with_structure = model.with_structured_output(json_schema, method="json_schema",)

response = model_with_structure.invoke("Provide details about the movie Inception")

print(type(response))
print(response) 

<class 'dict'>
{'title': 'Inception', 'year': 2010, 'director': 'Christopher Nolan', 'rating': 8.8}


### TypedDict

In [44]:
from typing_extensions import TypedDict, Annotated

class MovieDict(TypedDict):
    """A movie with details."""
    title: Annotated[str, ..., "The title of the movie"]
    year: Annotated[int, ..., "The year the movie was released"]
    director: Annotated[str, ..., "The director of the movie"]
    rating: Annotated[float, ..., "The movie's rating out of 10"]

model_with_structure = model.with_structured_output(MovieDict)

response = model_with_structure.invoke("Provide details about the movie Inception")

print(type(response))
print(response) 

<class 'dict'>
{'title': 'Inception', 'year': 2010, 'director': 'Christopher Nolan', 'rating': 8.8}


## Pydantic

In [45]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The movie's rating out of 10")

model_with_structure = model.with_structured_output(Movie)

response = model_with_structure.invoke("Provide details about the movie Inception")

print(type(response))
print(response)

<class '__main__.Movie'>
title='Inception' year=2010 director='Christopher Nolan' rating=8.8


In [46]:
model_with_structure.invoke(" film 'La vita è bella' del 1997  diretto da Roberto Benigni con voto 8.6 ")

Movie(title='La vita è bella', year=1997, director='Roberto Benigni', rating=8.6)

In [47]:
model_with_structure.invoke(" 'La vita è bella' , 1997, Roberto Benigni , otto ")

Movie(title='La vita è bella', year=1997, director='Roberto Benigni', rating=8.0)

## Callback Handler e Context Manager

### Esempio: conteggio token

In [24]:
# tramite callback handler

from langchain_core.callbacks import UsageMetadataCallbackHandler

model_1 = init_chat_model(model="openai:gpt-4o-mini")
model_2 = init_chat_model(model="openai:gpt-5-mini")

query = "Ciao, sono Enzo e mi piace l'arancione"

callback = UsageMetadataCallbackHandler()

# Un Callback Handler, in LangChain, è un oggetto che “si aggancia” (“hook”)
# ai vari eventi che accadono durante l’esecuzione di un modello, di una
# catena (chain), di un agente, ecc.
# L’obiettivo è intercettare azioni come “il modello sta per partire”,
# “il modello ha prodotto una risposta”, “c’è stato un errore”, ecc.,
# così da poter:
# - registrare metriche (es. token usati, tempo, costi)
# - tracciare i passaggi eseguiti
# - loggare input/output
# - effettuare streaming dei risultati o monitoraggio
# - ...(qualsiasi operazione necessitiamo)

# lista callback:
# https://reference.langchain.com/python/langchain_core/callbacks/

result_1 = model_1.invoke(query, config={"callbacks": [callback]})
result_2 = model_2.invoke(query, config={"callbacks": [callback]})

callback.usage_metadata

{'gpt-4o-mini-2024-07-18': {'input_tokens': 20,
  'output_tokens': 28,
  'total_tokens': 48,
  'input_token_details': {'audio': 0, 'cache_read': 0},
  'output_token_details': {'audio': 0, 'reasoning': 0}},
 'gpt-5-mini-2025-08-07': {'input_tokens': 19,
  'output_tokens': 947,
  'total_tokens': 966,
  'input_token_details': {'audio': 0, 'cache_read': 0},
  'output_token_details': {'audio': 0, 'reasoning': 512}}}

In [25]:
# tramite context manager

from langchain_core.callbacks import get_usage_metadata_callback

model_1 = init_chat_model(model="openai:gpt-4o-mini")
model_2 = init_chat_model(model="openai:gpt-5-mini")

with get_usage_metadata_callback() as cb:
    # La funzione get_usage_metadata_callback() restituisce un generatore/context-manager
    # che fornisce un handler interno e lo registra come callback per ogni chiamata al modello
    # nel blocco with. Alla fine, il campo cb.usage_metadata tiene traccia aggregata dell’uso
    # di token per tutti i modelli invocati nel blocco.
    # Vantaggi:
    # - Non devi passare manualmente il callback a ciascuna invoke.
    # - Raccoglie automaticamente tutti i modelli/chain eseguiti all’interno del blocco with.
    # - È più pulito quando hai più chiamate da tracciare in sequenza, senza ripetere configurazione.
    # - Alla uscita dal blocco viene garantita la “chiusura” del contesto (es. eventuali risorse
    #   liberate, handler deregistrato).
    #
    # È sicuramente più utile per comportamenti globali, non specifici di un singolo modello
    model_1.invoke(query)
    model_2.invoke(query)

print(cb.usage_metadata)

{'gpt-4o-mini-2024-07-18': {'input_tokens': 20, 'output_tokens': 36, 'total_tokens': 56, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}, 'gpt-5-mini-2025-08-07': {'input_tokens': 19, 'output_tokens': 668, 'total_tokens': 687, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 384}}}
